#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [1]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.2.0-py3-none-any.whl size=22721 sha256=b2c614e0f5811c38f4f4ff1e64dd1985c60df0c958cfdcf381a46eff828cd676
  Stored in directory: /tmp/pip-ephem-wheel-cache-kx9276aj/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.2.0
    Uninstalling boston311-0.2.0:
      Successfully uninstalled boston311-0.2.0


##Import the Boston311Model class

In [2]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import sys
import time

In [3]:
from boston311 import Boston311KerasNN


2023-10-30 19:24:14.183131: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 19:24:14.227890: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 19:24:14.228849: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 19:24:14.931622: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


## Load extra features

In [4]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

#format tomorrows date as yyyy-mm-dd
tomorrows_date =  now + pd.DateOffset(days=1)
tomorrows_datestring = tomorrows_date.strftime("%Y-%m-%d")

In [5]:
print('today_datestring: ', today_datestring)
print('tomorrows_datestring: ', tomorrows_datestring)

today_datestring:  2023-10-30
tomorrows_datestring:  2023-10-31


In [6]:
#set path to mydata
#EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_three_cols.csv'


In [7]:
import glob
import os

def get_latest_model_files(model_dir="./daily_models/Boston311KerasNN/"):
    # Get all json and h5 files
    json_files = glob.glob(os.path.join(model_dir, "*.json"))
    h5_files = glob.glob(os.path.join(model_dir, "*.h5"))

    # Sort files by modification time
    json_files.sort(key=os.path.getmtime, reverse=True)
    h5_files.sort(key=os.path.getmtime, reverse=True)

    if json_files and h5_files:
        latest_json = json_files[0]
        latest_h5 = h5_files[0]
        return latest_json, latest_h5
    else:
        return None, None

# Use the function
json_file, model_file = get_latest_model_files()
print(json_file)

if json_file and model_file:
    KerasNN_model = Boston311KerasNN()
    KerasNN_model.load(json_file, model_file)
    KerasNN_model.predict_date_range['end'] = tomorrows_datestring


./daily_models/Boston311KerasNN/20231030_192117_Boston311KerasNN.json


In [8]:
#load data from all_311_data.csv
mydata = pd.read_csv('all_311_cases.csv', low_memory=False)

In [10]:
data = KerasNN_model.load_data( data=mydata, train_or_predict='predict' )
data = KerasNN_model.enhance_data( data, 'predict')

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['closed_dt'] = pd.to_datetime(data['closed_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['open_dt'] = pd.to_datetime(data['open_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [13]:
#show first ten records with newest open_dt
data.sort_values(by=['open_dt'], ascending=False).head(10)

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,...,location_street_name,location_zipcode,latitude,longitude,geom_4326,source,survival_time,event,ward_number,survival_time_hours
2735253,101005146580,2023-10-29 19:19:00,2023-10-30 19:19:27,NaT,ONTIME,Open,,Traffic Signal Inspection,Transportation - Traffic Division,Signs & Signals,...,INTERSECTION Cambridge St & Brighton Ave,NaN,42.353581,-71.137334,0101000020E61000007FF63416CAC851C0A10F3B25422D...,Constituent Call,NaT,0,21,NaN
2735252,101005146578,2023-10-29 19:13:52,2023-11-02 04:30:00,NaT,ONTIME,Open,,Poor Conditions of Property,Public Works Department,Code Enforcement,...,55 S Huntington Ave,2130.0,42.330685,-71.111809,0101000020E6100000413D61DF27C751C00B386DDF532A...,Citizens Connect App,NaT,0,10,NaN
2735251,101005146577,2023-10-29 19:09:36,NaN,2023-10-29 19:10:13,ONTIME,Closed,Case Closed. Closed date : 2023-10-29 23:10:13...,Mattress Pickup,Public Works Department,Sanitation,...,274 Norfolk St,2124.0,42.283673,-71.081729,0101000020E61000002841DF0A3BC551C04CF8E4614F24...,Constituent Call,0 days 00:00:37,1,17,0.010278
2735250,101005146575,2023-10-29 19:01:30,2023-10-31 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,101 Condor St,2128.0,42.382393,-71.036411,0101000020E610000055A90E8E54C251C0276D4740F230...,Citizens Connect App,NaT,0,1,NaN
2735249,101005146574,2023-10-29 19:00:00,2023-10-31 04:30:00,NaT,ONTIME,Open,,Major System Failure,Public Works Department,Street Lights,...,3100 Washington St,2119.0,42.315002,-71.098879,0101000020E61000009C8AAE0754C651C0503161F95128...,Constituent Call,NaT,0,11,NaN
2735248,101005146572,2023-10-29 18:38:47,2023-10-31 04:30:00,NaT,ONTIME,Open,,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,2 Center Plz,2108.0,42.359799,-71.060202,0101000020E610000000CF7157DAC351C0D8906DE60D2E...,Citizens Connect App,NaT,0,3,NaN
2735247,101005146568,2023-10-29 18:24:19,2023-10-31 04:30:00,NaT,ONTIME,Open,,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,31 New Chardon St,2114.0,42.362380,-71.061320,0101000020E6100000B14F96ACECC351C03E60C673622E...,Citizens Connect App,NaT,0,3,NaN
2735246,101005146567,2023-10-29 18:18:03,2024-02-26 17:18:05,NaT,ONTIME,Open,,Pest Infestation - Residential,Inspectional Services,Housing,...,1218 VFW Pkwy,2132.0,42.278742,-71.170141,0101000020E6100000DB4A0697E3CA51C089BED4CDAD23...,Citizens Connect App,NaT,0,20,NaN
2735245,101005146566,2023-10-29 18:17:14,2024-01-04 03:30:00,NaT,ONTIME,Open,,Graffiti Removal,Property Management,Graffiti,...,INTERSECTION Anthony Rip Valenti Way & Portlan...,NaN,42.363653,-71.061000,0101000020E61000002EA4E56DE7C351C0BEF0E4308C2E...,Citizens Connect App,NaT,0,3,NaN
2735244,101005146565,2023-10-29 18:14:34,2023-10-31 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,36 Falcon St,2128.0,42.382119,-71.038190,0101000020E6100000E56AEEB571C251C0D0E0A24AE930...,Citizens Connect App,NaT,0,1,NaN


In [14]:
clean_data = KerasNN_model.clean_data_for_prediction( data )

In [16]:
mydata = clean_data

In [18]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    #check if the file date is earlier than EXTRA_mydata_FILE date
    pickle_file_date = os.path.getmtime(pickle_file)
    EXTRA_mydata_FILE_date = os.path.getmtime(EXTRA_mydata_FILE)
    if pickle_file_date < EXTRA_mydata_FILE_date:
        os.remove(pickle_file)

if os.path.exists(pickle_file):

    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    for col in ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']:
        X[col] = X[col].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [19]:
#concatenate the two dataframes and reindex
df = X

In [20]:
df.shape

(292876, 7)

In [21]:


# Assuming df is your DataFrame and it has columns 'desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding'
desc_cls_embedding_flattened = np.stack(df['desc_cls_embedding'].to_numpy())
desc_pooled_embedding_flattened = np.stack(df['desc_pooled_embedding'].to_numpy())
#do the same for the rest
name_cls_embedding_flattened = np.stack(df['name_cls_embedding'].to_numpy())
name_pooled_embedding_flattened = np.stack(df['name_pooled_embedding'].to_numpy())
code_cls_embedding_flattened = np.stack(df['code_cls_embedding'].to_numpy())
code_pooled_embedding_flattened = np.stack(df['code_pooled_embedding'].to_numpy())


# Remove the old columns
df.drop(['desc_cls_embedding', 'desc_pooled_embedding'], axis=1, inplace=True)
#do the same for the rest
df.drop(['name_cls_embedding', 'name_pooled_embedding'], axis=1, inplace=True)
df.drop(['code_cls_embedding', 'code_pooled_embedding'], axis=1, inplace=True)


# Add the new flattened columns
df_desc_cls = pd.DataFrame(desc_cls_embedding_flattened, columns=[f'desc_cls_{i}' for i in range(desc_cls_embedding_flattened.shape[1])])

df_desc_pooled = pd.DataFrame(desc_pooled_embedding_flattened, columns=[f'desc_pooled_{i}' for i in range(desc_pooled_embedding_flattened.shape[1])])
#do the same for the rest
df_name_cls = pd.DataFrame(name_cls_embedding_flattened, columns=[f'name_cls_{i}' for i in range(name_cls_embedding_flattened.shape[1])])
df_name_pooled = pd.DataFrame(name_pooled_embedding_flattened, columns=[f'name_pooled_{i}' for i in range(name_pooled_embedding_flattened.shape[1])])
df_code_cls = pd.DataFrame(code_cls_embedding_flattened, columns=[f'code_cls_{i}' for i in range(code_cls_embedding_flattened.shape[1])])
df_code_pooled = pd.DataFrame(code_pooled_embedding_flattened, columns=[f'code_pooled_{i}' for i in range(code_pooled_embedding_flattened.shape[1])])



df = pd.concat([df, df_desc_cls, df_desc_pooled, df_name_cls, df_name_pooled, df_code_cls, df_code_pooled], axis=1)

In [22]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()

In [23]:
df = df[is_numeric]

In [24]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [25]:
df.shape

(292876, 769)

In [26]:
mydata.shape

(23890, 241)

In [27]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')
#new_mydata = mydata


In [29]:
#new_mydata = new_mydata.merge(cyclical_df, on='case_enquiry_id', how='inner')

In [30]:
new_mydata.shape

(15706, 1009)

In [31]:

df = new_mydata

In [32]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64') 

In [33]:
#list the number of rows in X and y
print(df.dtypes)


case_enquiry_id              object
queue_ANML02_LostFound      float64
queue_ANML_General          float64
queue_BHA_General           float64
queue_BPD_Administrative    float64
                             ...   
code_pooled_123             float64
code_pooled_124             float64
code_pooled_125             float64
code_pooled_126             float64
code_pooled_127             float64
Length: 1009, dtype: object


In [34]:
#free all unused dataframes
#df_to_delete = [cls_embedding_flattened, pooled_embedding_flattened, df_cls, df_pooled, X, new_mydata, is_numeric, mydata]

#for data_frame in df_to_delete:
#    if data_frame is not None:
#        del data_frame

In [35]:
case_enquiry_id = df['case_enquiry_id']
X_predict = df.drop(['case_enquiry_id'], axis=1)


In [38]:

#parse CLS embedding column as array
predictions = KerasNN_model.model.predict(X_predict)

491/491 [==============================] - 1s 1ms/step


In [39]:
# Define a function to flatten an array into a string.
def array_to_string(arr):
    return ' '.join(map(str, arr))

# Apply the function along axis 1 (rows).
string_predictions = np.apply_along_axis(array_to_string, axis=1, arr=predictions)

# Now string_predictions is a 1D NumPy array where each element is a string
# that contains all the elements from the corresponding row in the original 2D array.
print(string_predictions)

['0.7924945 0.12694938 0.021496657 0.009263485 0.011357795 0.0055204467 0.00454192 0.0019179409 0.004208232 0.0031871377 0.0029626673 0.0072443206 0.0025089516 0.0008483814 0.000152816 0.00034751155 0.00021757308 0.00019321807 0.00016428603 0.00010538298 0.00019495406 0.00012015251 0.000500833 0.00022814143 5.88997e-05 0.00012470825 7.4262316e-05 4.4961816e-05 9.537917e-05 7.7849174e-05 0.00014651685 8.453825e-05 6.755215e-05 7.577671e-05 7.359472e-05 5.1034825e-05 0.0001956447 9.142142e-05 9.57922e-05 6.147046e-05 1.435525e-05 3.3957444e-05 3.9804683e-05 0.00010180342 4.250615e-05 2.8962086e-05 9.229349e-05 3.0114397e-05 8.402008e-05 2.788411e-05 2.7083515e-05 3.3914723e-05 9.321539e-05 4.872244e-05 5.814281e-05 6.785271e-05 7.0006456e-05 7.623656e-05 6.1211256e-05 9.0021575e-05 0.0007321233'
 '0.32161823 0.08061303 0.044663996 0.03550846 0.04068351 0.039077044 0.040905923 0.025100283 0.023998529 0.038305458 0.034642372 0.04169697 0.020840146 0.01381426 0.008601031 0.009360273 0.00838

In [40]:
#combine case_enquiry_id and predictions into a dataframe
predictions_df = pd.DataFrame({'case_enquiry_id':case_enquiry_id, 'prediction':string_predictions})

In [41]:
bin_labels = [
    "0-12 hours",      # Less than half a day
    "12-24 hours",     # Half to one day
    "1-3 days",        # One to three days
    "4-7 days",        # Four to seven days
    "1-2 weeks",       # One to two weeks
    "2-4 weeks",       # Two to four weeks
    "1-2 months",      # One to two months
    "2-4 months",      # Two to four months
    "4+ months"        # More than four months
]

#predictions_df['prediction'] = predictions_df['prediction'].apply(lambda x: bin_labels[x])

In [42]:
len(predictions_df)

15706

In [43]:
print(predictions)

[[7.9249448e-01 1.2694938e-01 2.1496657e-02 ... 6.1211256e-05
  9.0021575e-05 7.3212333e-04]
 [3.2161823e-01 8.0613032e-02 4.4663996e-02 ... 2.4236534e-03
  1.7534891e-03 1.5017914e-02]
 [9.9161625e-01 3.4891893e-03 1.2122089e-03 ... 5.7605510e-05
  6.9228736e-05 1.4941020e-04]
 ...
 [8.9067686e-01 3.9665081e-02 1.4345215e-02 ... 3.2859421e-04
  4.4111718e-04 1.5725308e-03]
 [7.7826738e-01 1.3323274e-01 2.3481537e-02 ... 6.4360473e-05
  8.6427601e-05 7.1258593e-04]
 [9.9769431e-01 1.2484137e-03 2.7749722e-04 ... 9.7458587e-06
  1.3010964e-05 2.8571745e-05]]


In [44]:
KerasNN_model.model_type

'Boston311KerasNN'

In [46]:
import pandas as pd

#get model_name from json_file name and ml_model_date from json_file name first 8 characters which are YYYYMMDD and change it to YYYY-MM-DD
model_name = json_file.split('/')[-1].split('.')[0]
ml_model_date = model_name[:4] + '-' + model_name[4:6] + '-' + model_name[6:8]

#define an empt pandas dataframe ml_model_df
ml_model_df = pd.DataFrame(columns=['ml_model_name', 'ml_model_type', 'ml_model_date'])

ml_model_df = pd.concat([ml_model_df, pd.DataFrame([{'ml_model_name': model_name, 
                                    'ml_model_type': KerasNN_model.model_type,
                                    'ml_model_date': ml_model_date}])], ignore_index=True)

print(ml_model_df)

                      ml_model_name     ml_model_type ml_model_date
0  20231030_192117_Boston311KerasNN  Boston311KerasNN    2023-10-30


In [48]:
model_cases = data.drop(['geom_4326','survival_time_hours', 'survival_time', 'event'], axis=1).copy()

In [49]:
all_model_cases = model_cases 
all_model_predictions = predictions_df

In [50]:
all_model_predictions['ml_model_name'] = model_name

In [51]:
all_model_predictions['prediction_date'] = today_datestring

In [52]:
# %%
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [53]:


# %%
all_model_cases.to_csv(my_datetime+'_311_cases.csv', index=False)


# %%

all_model_predictions.to_csv(my_datetime+'_311_predictions.csv', index=False)

# %%

ml_model_df.to_csv(my_datetime+'_311_ml_models.csv', index=False)

# %%
#create datetime _manifest.txt file with one filename per line
with open(my_datetime+'_manifest.txt', 'w') as f:
    f.write(my_datetime+'_311_cases.csv\n')
    f.write(my_datetime+'_311_predictions.csv\n')
    f.write(my_datetime+'_311_ml_models.csv\n')

# %%
#create an export folder
EXPORT_FOLDER = '~/Documents/BODC-DEI-site/database/seeders'
#copy the csv files to the export folder
!cp {my_datetime}_311_cases.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_predictions.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_ml_models.csv {EXPORT_FOLDER}
!cp {my_datetime}_manifest.txt {EXPORT_FOLDER}



# %% [markdown]
# ** Copy the files to the production server **

# %%
# Define constants for servers
PROD_USER = 'u353344964'
PROD_HOSTNAME = '195.179.236.61'
PORT_NUMBER = 65002
PROD_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel'
STAGE_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel'
PROD_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel/database/seeders'
STAGE_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders'

# %%







In [54]:
import os


def scp_to_server(filename, user=PROD_USER, hostname=PROD_HOSTNAME, port=PORT_NUMBER, export_folder=PROD_EXPORT_FOLDER):
    """Copy a file to the server using scp."""
    command = f"scp -P {port} {filename} {user}@{hostname}:{export_folder}"
    print(f"Executing: {command}")
    os.system(command)

# Use the function to scp files
files_to_copy = [
    f"{my_datetime}_311_cases.csv",
    f"{my_datetime}_311_predictions.csv",
    f"{my_datetime}_311_ml_models.csv",
    f"{my_datetime}_manifest.txt"
]

# Control where to copy
copy_to_prod = True
copy_to_stage = True

for file in files_to_copy:
    if copy_to_prod:
        scp_to_server(file, export_folder=PROD_EXPORT_FOLDER)
    if copy_to_stage:
        scp_to_server(file, export_folder=STAGE_EXPORT_FOLDER)


# %%
PORT_NUMBER

Executing: scp -P 65002 2023-10-30_19-28-01_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders


Executing: scp -P 65002 2023-10-30_19-28-01_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-30_19-28-01_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-30_19-28-01_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-30_19-28-01_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-30_19-28-01_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-30_19-28-01_manifest.txt u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-30_19-28-01_manifest.txt u353344964@195.179.236.61:/

65002

In [55]:

if copy_to_prod:
    ! ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {PROD_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'

if copy_to_stage:
    ! ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {STAGE_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'


   INFO  Seeding database.  


Manifest files:
/home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-30_19-28-01_manifest.txt

Total records to process: 39597

Processing /home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-30_19-28-01_311_cases.csv
100 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 23790.
Total records remaining: 39497.
Time for last 100 records: 0.02 seconds.
Estimated time remaining for this file: 4 seconds.
Estimated time for all files: 8 seconds.
200 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 23690.
Total records remaining: 39397.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 1 second.
Estimated time for all files: 2 seconds.
300 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 23590.
Total records remaining: 39297.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 1 sec